In [1]:
# Change the dataset in global_vars.py to KORA.
from global_vars import *
from commons import *

import glob 
import os

one_time_n4_optimization = True
vol_to_check_list = None # ['KORA2460830', 'KORA2460249']
exclude = ['KORA2460249']

%load_ext autoreload
%autoreload 2

In [2]:
# os.listdir(label_dir)

In [3]:
def load_kora_file_paths(load_from_txt_file=True):
    volumes_to_use = []
    if load_from_txt_file:
        with open(volume_txt_file) as file_handle:
                volumes_to_use = file_handle.read().splitlines()
    else:
        volumes_to_use = [name for namelabel in os.listdir(data_dir)]
    
    file_paths = {}
    
    for vol in volumes_to_use:
        if (vol_to_check_list is not None and vol not in vol_to_check_list) or (vol == "") or (vol in exclude):
            continue
            
        labelmap_paths = glob.glob(f'{label_dir}/{vol}/**')
        opp_paths = glob.glob(f'{data_dir}/{vol}/**opp_[0-9]**.nii.gz')
        in_paths = glob.glob(f'{data_dir}/{vol}/**in_[0-9]**.nii.gz')
        f_paths = glob.glob(f'{data_dir}/{vol}/**F_[0-9]**.nii.gz')
        w_paths = glob.glob(f'{data_dir}/{vol}/**W_[0-9]**.nii.gz')
        
        vol_madals_paths = dict(
        OPP=opp_paths,
        IN=in_paths,
        F=f_paths,
        W=w_paths
        )
        file_paths[str(vol)]=dict(
            VOLUME_PATHS=vol_madals_paths,
            LABEL_PATHS=labelmap_paths,
        )
    return file_paths

file_paths = load_kora_file_paths()
file_paths

{'KORA2457401': {'VOLUME_PATHS': {'OPP': ['/mnt/nas/Data_WholeBody/KORA/KORA_all/KORA_Nifti/KORA2457401/t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8.nii.gz',
    '/mnt/nas/Data_WholeBody/KORA/KORA_all/KORA_Nifti/KORA2457401/t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_9.nii.gz'],
   'IN': ['/mnt/nas/Data_WholeBody/KORA/KORA_all/KORA_Nifti/KORA2457401/t1_vibe_dixon_cor_caipi6_bh_288_iso_in_6.nii.gz',
    '/mnt/nas/Data_WholeBody/KORA/KORA_all/KORA_Nifti/KORA2457401/t1_vibe_dixon_cor_caipi6_bh_288_iso_in_7.nii.gz'],
   'F': ['/mnt/nas/Data_WholeBody/KORA/KORA_all/KORA_Nifti/KORA2457401/t1_vibe_dixon_cor_caipi6_bh_288_iso_F_1.nii.gz',
    '/mnt/nas/Data_WholeBody/KORA/KORA_all/KORA_Nifti/KORA2457401/t1_vibe_dixon_cor_caipi6_bh_288_iso_F_2.nii.gz'],
   'W': ['/mnt/nas/Data_WholeBody/KORA/KORA_all/KORA_Nifti/KORA2457401/t1_vibe_dixon_cor_caipi6_bh_288_iso_W_3.nii.gz',
    '/mnt/nas/Data_WholeBody/KORA/KORA_all/KORA_Nifti/KORA2457401/t1_vibe_dixon_cor_caipi6_bh_288_iso_W_4.nii.gz']},
  'LABEL_PATHS'

In [4]:
# Individual RESCALING.
n4_dict = {}
for vol in file_paths.keys():
    if one_time_n4_optimization:
        break
    print(f'n4 processing part-1 started with {vol}...')
    n4_dict[vol] = []
    vol_parts = [[file, read_ras(file)] for file in file_paths[vol]['VOLUME_PATHS']['IN']]
    for orig_file, in_image in vol_parts:
        n4_dict[vol].append(rescale(in_image, vol, orig_file))

In [5]:
# SITK does ot work due to differences in pixel resolution of IN and corresponding OPP Scan.
# Only applying once at the end.
for vol in file_paths.keys():
    if one_time_n4_optimization:
        break
    if vol_to_check_list is not None and vol not in vol_to_check_list:
        continue
    print(f'n4-biasfield-correction starting with {vol}...')
    for idx, n4_d in enumerate(n4_dict[vol]):
        in_file = n4_d['SCALED']
        opp_file = file_paths[vol]['VOLUME_PATHS']['OPP'][idx]
        new_filename = opp_file.split('/')[-1].split('.')[0]
        output_file = f'{n4_corrected_data_dir}/vol/{vol}/{new_filename}_n4_corrected_sitk.nii.gz'
        SITK_N4_normalization(in_file, opp_file, output_file)
        n4_dict[vol][idx]['OPP_CORRECTED'] = output_file

    file_paths[vol]['N4_1'] = n4_dict[vol]

In [ ]:
# STITCHING VOL PARTS HERE
for vol in file_paths.keys():
    try:
        if vol_to_check_list is not None and vol not in vol_to_check_list:
            continue
        print(f'started with {vol}...')
        create_if_not(f'{n4_corrected_data_dir}/vol/{vol}')
        file_paths[vol]['ONE'] = {}
        for modality_key in file_paths[vol]['VOLUME_PATHS'].keys():
            print(f"processing {modality_key}")
            orig_modal_key = modality_key
            if one_time_n4_optimization:
                vol_parts = [read_ras(file) for file in file_paths[vol]['VOLUME_PATHS'][modality_key]]
            else:
                if modality_key == 'OPP':
                    vol_parts = [read_ras(data_dict['OPP_CORRECTED']) for data_dict in file_paths[vol]['N4_1']]
                    modality_key = modality_key+'_n4_corrected'
                else:
                    vol_parts = [read_ras(file) for file in file_paths[vol]['VOLUME_PATHS'][modality_key]]

            ras_stitched_resampled = multi_vol_stitching(vol_parts)
            save_volume(ras_stitched_resampled, f'{n4_corrected_data_dir}/vol/{vol}/{modality_key}_ras_stitched')
            file_paths[vol]['ONE'][f'{orig_modal_key}'] = f'{n4_corrected_data_dir}/vol/{vol}/{modality_key}_ras_stitched.nii.gz'
    except Exception as e:
        print(e)
        continue

started with KORA2457401...
processing OPP
Reading Files.....
Reading Nifti Files.....
Affine:[[ -1.70138884   0.           0.         249.86656189]
 [  0.           0.          -1.69999695 164.43530273]
 [  0.          -1.70138884   0.         -33.90978622]
 [  0.           0.           0.           1.        ]], Image Shape: (288, 288, 176)
Transforming Images to RAS.....
Reading Files.....
Reading Nifti Files.....
Affine:[[  -1.66666663    0.            0.          244.86656189]
 [   0.            0.           -1.69999695  164.43530273]
 [   0.           -1.66666663    0.         -268.90979004]
 [   0.            0.            0.            1.        ]], Image Shape: (288, 288, 176)
Transforming Images to RAS.....
0th img for stitching...
saving directory: /mnt/nas/Abhijit/Jyotirmay/abdominal_segmentation/temp/KORA/n4_corrected_2/vol/KORA2457401
processing IN
Reading Files.....
Reading Nifti Files.....
Affine:[[ -1.70138884   0.           0.         249.86656189]
 [  0.           0.

Transforming Images to RAS.....
0th img for stitching...
saving directory: /mnt/nas/Abhijit/Jyotirmay/abdominal_segmentation/temp/KORA/n4_corrected_2/vol/KORA2456340
processing F
Reading Files.....
Reading Nifti Files.....
Affine:[[ -1.70138884   0.           0.         235.26948547]
 [  0.           0.          -1.70000458 104.78657532]
 [  0.          -1.70138884   0.          -1.61030149]
 [  0.           0.           0.           1.        ]], Image Shape: (288, 288, 160)
Transforming Images to RAS.....
Reading Files.....
Reading Nifti Files.....
Affine:[[  -1.66666663    0.            0.          230.26948547]
 [   0.            0.           -1.70000458  104.78657532]
 [   0.           -1.66666663    0.         -236.61030579]
 [   0.            0.            0.            1.        ]], Image Shape: (288, 288, 160)
Transforming Images to RAS.....
0th img for stitching...
saving directory: /mnt/nas/Abhijit/Jyotirmay/abdominal_segmentation/temp/KORA/n4_corrected_2/vol/KORA2456340
pro

In [ ]:
# RESCALING INTENSITIES OF STITCHED VOLUME ABOVE 0
n4_dict = {}
for vol in file_paths.keys():
    if vol_to_check_list is not None and vol not in vol_to_check_list:
        continue
    print(f'n4 processing part-2 started with {vol}...')
    n4_dict[vol] = {}
    in_stitched_file_path, in_stitched_img = file_paths[vol]['ONE']['IN'], read_ras(file_paths[vol]['ONE']['IN'])
    n4_dict[vol]['N4_2'] = rescale(in_stitched_img, vol, in_stitched_file_path)

In [ ]:
for vol in file_paths.keys():
    if vol_to_check_list is not None and vol not in vol_to_check_list:
        continue
    print(f'n4-biasfield-correction starting with {vol}...')
    in_file = n4_dict[vol]['N4_2']['SCALED']
    opp_file = file_paths[vol]['ONE']['OPP']
    new_filename = opp_file.split('/')[-1].split('.')[0]
    output_file = f'{n4_corrected_data_dir}/vol/{vol}/{new_filename}_n4_corrected_sitk.nii.gz'
    SITK_N4_normalization(in_file, opp_file, output_file)
    n4_dict[vol]['N4_2']['OPP_CORRECTED'] = output_file
    file_paths[vol]['N4_2'] = n4_dict[vol]['N4_2']

In [ ]:
def kora_vol_label_fix(vol, label, use_alternate_approach=False):
    world_shape = np.max(np.array([list(vol.shape), list(label.shape)]), axis=0)
    final_label = np.zeros(tuple(world_shape))
    
    label_affine = label.affine
    vol_affine = vol.affine
    target_affine = vol_affine
    target_header = vol.header

    sx,sy,sz,ex,ey,ez = np.abs(get_points(label, vol))

    ld = label.get_fdata()
    if not use_alternate_approach:
        final_label[0:sx+ex, 0:sy+ey, 0:sz+ez] = ld
    else:
        final_label[0:sx+ex, 0:sy+ey, 77:77+76] = ld[:, :, :76]
        
    final_label = np.flip(final_label, axis=0)
    final_label = np.flip(final_label, axis=2)
    
    final_label_img = nb.Nifti1Image(final_label, target_affine, target_header)
    
    return vol, final_label_img

def kora_label_parts(label_parts, reference_labelmap=None):
    stitched_label = None
    mode = 'constant'
    order = 0
    if reference_labelmap is None:
        label_shape = get_freequent_shape([img.shape for img, _, _ in label_parts])
        reference_labelmap = [img for img, _, _ in label_parts if list(img.shape) == list(label_shape)][0]
    else:
        label_shape = reference_labelmap.shape
        
    stitched_label = np.zeros(label_shape)
    for labelmap_img, lidx, lname in label_parts:
        print(lidx, lname)
        labelmap_img = makeit_3d(labelmap_img)
        labelmap_img = resample_from_to(labelmap_img, [label_shape, reference_labelmap.affine], order=order, mode=mode, cval=0)
        
        sx,sy,sz,ex,ey,ez = np.abs(get_points(labelmap_img, reference_labelmap))
        
        labelmap = labelmap_img.get_fdata()
        labelmap = np.multiply(lidx, labelmap)
        stitched_label[0:ex+sx, 0:ey+sy, 0:ez+sz] += labelmap
        
        print("###############################################################################################") 
        
    labelmap = np.round(stitched_label)
    stitched_labeled_img = nb.Nifti1Image(labelmap, reference_labelmap.affine, reference_labelmap.header)
    
    return stitched_labeled_img

In [ ]:
# import pickle as p
# with open('new_oksana_kora_file_paths.p', 'wb') as handle:
#     p.dump(file_paths, handle, protocol=p.HIGHEST_PROTOCOL)

In [ ]:
# import pickle as p
# with open('new_oksana_kora_file_paths.p', 'rb') as handle:
#     file_paths = p.load(handle)

In [ ]:
# exclude = ['KORA2460249', 'KORA2456120'] #, 'KORA2456120',  'KORA2462026']
# vol_to_check_list = None #['KORA2452879', 'KORA2456120']
print("STARTING KORA LABEL-MAPS.")
print('Reading Label Maps.....')
skip = True
for vol in file_paths.keys():
    print(vol)
    if (vol_to_check_list is not None and vol not in vol_to_check_list) or (vol == "") or (vol in exclude):
            continue
#     if vol == 'KORA2462026':
#         skip = False
#     if skip:
#         continue
    later = []
    print(file_paths[vol]['LABEL_PATHS'])
    if len(file_paths[vol]['LABEL_PATHS']) == 0:
        print(f"#################### ALERT:: NO LABELPATHS IN THE DICTIONARY FOR {vol} #########################")
        continue
    volume = nb.load(file_paths[vol]['N4_2']['OPP_CORRECTED'])
#     ref_vol = nb.load(file_paths[vol]['ONE_ORIG']['OPP'])
    img_ras_list = []
    for label_file_to_read in file_paths[vol]['LABEL_PATHS']:
        img_ras, lidx, labelname = read_ras(label_file_to_read, is_label=True)
        if img_ras is None:
            continue
        img_ras = makeit_3d(img_ras)
        if labelname in ['PANCREAS']:
            later.append([img_ras, lidx+LABEL_EXTENSION_FOR_OVERLAP_REMOVAL, labelname])
        else:
            img_ras_list.append([img_ras, lidx+LABEL_EXTENSION_FOR_OVERLAP_REMOVAL, labelname])
    
    img_ras_list.extend(later)
    
    s_label = kora_label_parts(img_ras_list)
    s_label = resample_to_output(s_label, TARGET_RESOLUTION, order=0, mode='constant', cval=0)
    s_label = drop_overlapped_pixels(s_label, np.array(img_ras_list)[:, 1])
    
    if vol == 'KORA2460249':
        volume, s_label = kora_vol_label_fix(volume, s_label, True)
    else:
        volume, s_label = kora_vol_label_fix(volume, s_label)
    
    print('Viewing Stitched Images.....')
    volume_3_view_viewer(get_volume_data(volume))
    volume_3_view_viewer(get_volume_data(s_label))

    print('Saving Processed & Stitched Image.....')
    save_volume(volume, f'{processed_dir}/volume/{vol}')
    save_volume(s_label, f'{processed_dir}/label/{vol}')
    
print('FINISHED.')
